In [ ]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 65.8 MB/s eta 0:00:00


In [ ]:
import os
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, box
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
import pandas as pd

# ----- Configuration -----
lineament_path = r"/content/lineament/Lineament_tectonics.shp"  # Update path
output_tif = "lineament_decay_buffer.tif"

# Set bounding box in Lat/Lon
xmin, ymin = 75.99999, 13.75
xmax, ymax = 76.75, 14.5

# ~30m resolution in degrees
res_deg = 0.0002695
width = int((xmax - xmin) / res_deg)
height = int((ymax - ymin) / res_deg)

# ----- Load Lineament Shapefile -----
gdf = gpd.read_file(lineament_path).to_crs("EPSG:4326")

# ----- Generate 2D Grid -----
x_coords = np.linspace(xmin + res_deg / 2, xmax - res_deg / 2, width)
y_coords = np.linspace(ymax - res_deg / 2, ymin + res_deg / 2, height)
xx, yy = np.meshgrid(x_coords, y_coords)
pixel_points = np.vstack([xx.ravel(), yy.ravel()]).T

# ----- Compute Minimum Distance to Any Lineament -----
from shapely.geometry import Point
min_distances = np.full(pixel_points.shape[0], np.inf)

for geom in gdf.geometry:
    dists = np.array([geom.distance(Point(xy)) for xy in pixel_points])
    min_distances = np.minimum(min_distances, dists)

# ----- Convert Distance to Rank -----
# Distances in degrees: 1km ≈ 0.009°, 5km ≈ 0.045°
ranks = np.where(
    min_distances <= 0.009, 1.0,                         # <=1 km
    np.where(min_distances <= 0.045,
             1 - (min_distances - 0.009) / (0.045 - 0.009),  # 1 to 5 km: linear decay
             0.0)                                         # >5 km
)

# Reshape to raster
rank_raster = ranks.reshape((height, width))
rank_raster[rank_raster == 0.0] = np.nan  # Set NoData outside 5km

# ----- Save Raster -----
transform = from_origin(xmin, ymax, res_deg, res_deg)

with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype='float32',
    crs='EPSG:4326',
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(rank_raster.astype(np.float32), 1)

print(f"Saved lineament proximity decay raster to: {output_tif}")


Saved lineament proximity decay raster to: lineament_decay_buffer.tif
